In [2]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
db = client.eldenring
collection = db.weaponsummary

In [5]:
# URL of page to be scraped
url = 'https://eldenring.wiki.fextralife.com/Weapons+Comparison+Tables'
# retrieve page with the reqests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html5lib')

In [11]:
thead = soup.find('thead')
ths = thead.find_all('th')
keys = []
for th in ths:
    if th.text.strip() not in keys:
        keys.append(th.text.strip())
    else:
        keys.append("def-" + th.text.strip())
print(keys)

['Weapon', 'Type', 'Phy', 'Mag', 'Fir', 'Lit', 'Hol', 'Cri', 'Sta', 'Str', 'Dex', 'Int', 'Fai', 'Arc', 'Any', 'def-Phy', 'def-Mag', 'def-Fir', 'def-Lit', 'def-Hol', 'Bst', 'Rst', 'Wgt', 'Upgrade']


In [16]:
# create function for replacing values for cleaning the data
def replace_values(list_to_replace, item_to_replace, item_to_replace_with):
    return [item_to_replace_with if item == item_to_replace else item for item in list_to_replace]

In [33]:
tbody = soup.find('tbody')
trs = tbody.find_all('tr')
for tr in trs:
    data = []
    post ={}
    tds = tr.find_all('td')
    for td in tds:
        data.append(td.text)
        data = replace_values(data, "-", "0")
    for key in keys:
        for value in data:
            post[key] = value
            data.remove(value)
            break
    int_values = ['Bst','Cri','Fir','Hol','Lit','Mag','Phy','Rst','Sta','Wgt']
    for v in int_values:
        post[v] = float(post[v])
    collection.insert_one(post)
    print(post)

{'Weapon': 'Academy Glintstone Staff', 'Type': 'Glintstone Staff', 'Phy': 43.0, 'Mag': 0.0, 'Fir': 0.0, 'Lit': 0.0, 'Hol': 0.0, 'Cri': 100.0, 'Sta': 40.0, 'Str': 'D', 'Dex': '0', 'Int': 'S', 'Fai': '0', 'Arc': '0', 'Any': '0', 'def-Phy': '25', 'def-Mag': '15', 'def-Fir': '15', 'def-Lit': '15', 'def-Hol': '15', 'Bst': 15.0, 'Rst': 10.0, 'Wgt': 3.0, 'Upgrade': 'Smithing Stones', '_id': ObjectId('62530f1b299ed48c0fac746a')}
{'Weapon': "Alabaster Lord's Sword", 'Type': 'Greatsword', 'Phy': 313.0, 'Mag': 93.0, 'Fir': 0.0, 'Lit': 0.0, 'Hol': 0.0, 'Cri': 100.0, 'Sta': 126.0, 'Str': 'B', 'Dex': 'D', 'Int': 'D', 'Fai': '0', 'Arc': '0', 'Any': '0', 'def-Phy': '41', 'def-Mag': '33', 'def-Fir': '27', 'def-Lit': '27', 'def-Hol': '27', 'Bst': 33.0, 'Rst': 19.0, 'Wgt': 8.0, 'Upgrade': 'Somber Smithing Stones', '_id': ObjectId('62530f1b299ed48c0fac746b')}
{'Weapon': 'Albinauric Bow', 'Type': 'Bow', 'Phy': 200.0, 'Mag': 0.0, 'Fir': 0.0, 'Lit': 0.0, 'Hol': 0.0, 'Cri': 100.0, 'Sta': 60.0, 'Str': 'E', 'De

In [ ]:
collection.drop("Weapon":)